In [561]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [562]:
from google.colab.patches import cv2_imshow

In [563]:
def process_image(image, 
                  kernel_size = 9, 
                  low_threshold = 20, high_threshold = 30, 
                  rho = 1, theta = np.pi/180, threshold = 30,
                  min_line_len = 10, max_line_gap = 200 , past_line_mean = []): 
    flag = np.array(past_line_mean)

    #Change Image to GrayScale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    #Use Gaussian Filter
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    #Use Canny with Image Smoothing
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    #Find Region of Interest
    vertices = np.array([[(0,image.shape[0]),(725, 415), (745, 415), (image.shape[1],image.shape[0])]], dtype=np.int32)
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255 
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(edges, mask)

    #Use HoughLinesP
    img = masked_image.copy()

    lines = cv2.HoughLinesP(masked_image, rho, theta, threshold, minLineLength=min_line_len, maxLineGap=max_line_gap)
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

    #Calculate Means
    line_img = np.zeros((masked_image.shape[0], masked_image.shape[1], 3), dtype=np.uint8)

    color=[0, 0, 255]
    thickness=5

    x_bottom_pos = []
    x_upperr_pos = []
    x_bottom_neg = []
    x_upperr_neg = []

    y_bottom = 800
    y_upperr = 430

    slope = 0
    b = 0

    for line in lines:
        for x1,y1,x2,y2 in line:
            #test and filter values to slope
            if ((y2-y1)/(x2-x1)) > 0 :
                
                slope = ((y2-y1)/(x2-x1))
                b = y1 - slope*x1
                
                x_bottom_pos.append((y_bottom - b)/slope)
                x_upperr_pos.append((y_upperr - b)/slope)
                                      
            elif ((y2-y1)/(x2-x1)) < 0:
            
                slope = ((y2-y1)/(x2-x1))
                b = y1 - slope*x1
                
                x_bottom_neg.append((y_bottom - b)/slope)
                x_upperr_neg.append((y_upperr - b)/slope)

    if not x_bottom_pos or not x_upperr_pos:
      pos_side = past_line_mean[0]
    else :
      pos_side = [int(np.mean(x_bottom_pos)), int(np.mean(y_bottom)), int(np.mean(x_upperr_pos)), int(np.mean(y_upperr))]

    if not x_bottom_neg or not x_upperr_neg:
      neg_side = past_line_mean[1]
    else :
      neg_side = [int(np.mean(x_bottom_neg)), int(np.mean(y_bottom)), int(np.mean(x_upperr_neg)), int(np.mean(y_upperr))]

    if flag.any() :
      pos_side = [int(np.mean([pos_side[0] , past_line_mean[0][0]])) , int(np.mean(y_bottom)) , int(np.mean([pos_side[2] , past_line_mean[0][2]])) , int(np.mean(y_upperr))]
      neg_side = [int(np.mean([neg_side[0] , past_line_mean[1][0]])) , int(np.mean(y_bottom)) , int(np.mean([neg_side[2] , past_line_mean[1][2]])) , int(np.mean(y_upperr))]

    #creating a new 2d array with means
    lines_mean = np.array([pos_side , neg_side])


    #Drawing the lines
    for i in range(len(lines_mean)):
        cv2.line(line_img, (lines_mean[i,0], lines_mean[i,1]), (lines_mean[i,2], lines_mean[i,3]), color, thickness)

    # initial_img * α + img * β + λ
    lines_edges = cv2.addWeighted(image, 0.8, line_img, 1., 0)
    
    return lines_edges , lines_mean

In [564]:
video_cap = cv2.VideoCapture("vid2.mp4")

In [565]:
fps = video_cap.get(cv2.CAP_PROP_FPS)
height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FOURCC = cv2.VideoWriter_fourcc(*'MP4V')

In [566]:
frames = []
while True:
    success, frame = video_cap.read()
    if success:
      frames.append(frame)
    else:
        break
video_cap.release()
cv2.destroyAllWindows()

In [567]:
print(fps , height , width , FOURCC , len(frames))

30.0 720 1280 1446269005 319


In [568]:
out = cv2.VideoWriter('vid2_line_detected.mp4', int(FOURCC), fps, (width,height))

In [569]:
lines_mean = []
for i , image in enumerate(frames):
  # print(str(i))
  img , lines_mean = process_image(image , past_line_mean=lines_mean)
  out.write(img)
out.release()